# Cosine Similarity - Near Search

- Perform a near search using cosine similarity

### Importt packages

In [65]:
#r "nuget: dotenv.net"

using System;
using System.Net.Http;

using dotenv.net;
DotEnv.Load();

var apiKey=Environment.GetEnvironmentVariable("OPENAI_KEY");
var adaEndpoint=Environment.GetEnvironmentVariable("OPENAI_ADA_FULLURI");

record Data(string @object, int index, double[] embedding);
record EmbeddingResponse(string @bject, Data[] data);

HttpClient client = new();
// Json content
client.DefaultRequestHeaders.TryAddWithoutValidation("Content-Type", "application/json");
client.DefaultRequestHeaders.TryAddWithoutValidation("api-key", apiKey);


Installed Packages dotenv.net, 3.1.3

### Get an ADA text embedding

In [66]:
async Task<double[]> GetEmbedding(string input) 
{
    var content = new { input = input};
    var json = System.Text.Json.JsonSerializer.Serialize(content);
    var stringContent = new StringContent(json, System.Text.Encoding.UTF8, "application/json");    
    var request = await client.PostAsync(adaEndpoint, stringContent);
    if (request.IsSuccessStatusCode)
    {
        var response = await request.Content.ReadAsStringAsync();
        var adares = System.Text.Json.JsonSerializer.Deserialize<EmbeddingResponse>(response);
        return adares.data[0].embedding;
    }
    Console.WriteLine(request.StatusCode);
    return [];
}

### Calculate the Cosine Similarity

In [67]:
double CosineSimilarity(double[] vectorA, double[] vectorB)
{
    double dotProduct = 0.0;
    double magnitudeA = 0.0;
    double magnitudeB = 0.0;
    for (int i = 0; i < vectorA.Length; i++)
    {
        dotProduct += (vectorA[i] * vectorB[i]);
        magnitudeA += Math.Pow(vectorA[i], 2);
        magnitudeB += Math.Pow(vectorB[i], 2);
    }
    magnitudeA = Math.Sqrt(magnitudeA);
    magnitudeB = Math.Sqrt(magnitudeB);
    if (magnitudeA != 0 && magnitudeB != 0)
    {
        return dotProduct / (magnitudeA * magnitudeB);
    }
    else
    {
        return 0.0;
    }
}

### Prepare the mock vector database

In [72]:
List<string> content = [
    "The chemical composition of water is H2O.",
    "The speed of light is 300,000 km/s.",
    "Acceleration of gravity on earth is 9.8m/s^2.",
    "The chemical composition of salt or sodium clorida is NaCl.",
];

List<Tuple<string, double[]>> vectorDb = new();
foreach (var item in content)
{
    var embedding = await GetEmbedding(item);
    vectorDb.Add(new Tuple<string, double[]>(item, embedding));
}

### Embed the question

In [73]:
var input = "What is the speed of light?";
var e1 = await GetEmbedding(input);


### Perform Nearest search

In [74]:
const int limit=3;
const double relevance=0.5;
var count = 0;
List<Tuple<string, double[]>> resultsList = new();
foreach(var item in vectorDb)
{
    var sim = CosineSimilarity(e1, item.Item2);
    if(sim>relevance)
    {
        //Console.WriteLine($"{item.Item1} - {sim}");
        resultsList.Add(item);
        count++;
    }
    if(count>=limit)
    {
        break;
    }
}

### Print Results

In [75]:
resultsList = resultsList.OrderByDescending(x => CosineSimilarity(e1, x.Item2)).ToList();
foreach(var item in resultsList)
{
    Console.WriteLine($"Similarity: {CosineSimilarity(e1, item.Item2)}, Content: {item.Item1}");
}

Similarity: 0.9076484494720922, Content: The speed of light is 300,000 km/s.
Similarity: 0.8044927978430234, Content: Acceleration of gravity on earth is 9.8m/s^2.
Similarity: 0.7380432345349796, Content: The chemical composition of water is H2O.
